In [15]:
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    temperature=1,
    model_name="gpt-5-nano",
    streaming=True,
    )

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
 embeddings, cache_dir   
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the follwing context."
    "If you don't knwo the answer just say you don't know, don't make it up:\n{context}"),
    ("human","{question}")
])

# list of docs

# for doc in list of docs | prompt | llm

# for respone lin list of llms response | put them all together
    
#     final doc | prompt | llm 

map_doc_prompt = ChatPromptTemplate.from_messages(
       [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content,"question": question}
        ).content
        for doc in documents
    )

map_chain = {
    "documents": retriever,
      "question":RunnablePassthrough()
      } | RunnableLambda(map_docs)


final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = { "context": map_chain,"question":RunnablePassthrough()} | final_prompt | llm

chain.invoke("Describe Victory Mansions")

AIMessageChunk(content='Victory Mansions is the dilapidated block of flats in Airstrip One (formerly Britain) where Winston Smith lives in George Orwell’s 1984. It’s a tall, gray, aging building that embodies the regime’s deceptive sense of progress.\n\n- Exterior and building: The block looks decent from the outside but is run-down inside—cracked walls, damp plaster, and a stale, oppressive atmosphere. The corridors are dingy, and the lifts are unreliable.\n- Inside the flat: Winston’s apartment is small and sparsely furnished, with a bare, functional setup designed for practicality rather than comfort. The air is stale and the room feels cramped.\n- Surveillance: A telescreen dominates the living space, constantly watching and listening, underscoring the Party’s omnipresence.\n- Sensory details: The environment smells faintly of boiled cabbage and dampness, a dull and oppressive mood pervading the space.\n- Irony of the name: The very name “Victory Mansions” is propaganda—an ironic c